In [1]:
import sys
sys.path.append('../src/')

from adaline import Adaline
from data_loader import DataLoader
from operator import and_, or_, xor
from random_fns import Uniform
from simple_perceptron import SimplePerceptron
from trainer import Trainer

import numpy as np

In [2]:
def generate_binary_fn(fn, logic='bipolar') -> list[tuple[np.ndarray, np.ndarray]]:
    from itertools import product

    f = lambda x: 1.0 if x else (-1.0 if logic == 'bipolar' else 0.0)
    data = [(np.array([[f(a), f(b)]]), np.array([[f(fn(a, b))]]))
            for a in [False, True] for b in [False, True]]
    return data

def augment_data(data, n) -> list[tuple[np.ndarray, np.ndarray]]:
    augmented = []
    for x, y in data:
        for _ in range(n):
            augmented_x = x.copy()
            augmented_x[0][0] += (np.random.rand() - 0.5) / 100
            augmented_x[0][1] += (np.random.rand() - 0.5) / 100
            augmented.append((augmented_x, y))
    return augmented

In [3]:
models = [
    SimplePerceptron(
        input_size=2,
        random_fn=Uniform(-1.0, 1.0),
        learning_rate=0.01,
    ),
    Adaline(
        input_size=2,
        random_fn=Uniform(-1.0, 1.0),
        learning_rate=0.01,
    )
]

In [4]:
data = generate_binary_fn(and_)

train_dataloader = DataLoader(data)
val_dataloader = DataLoader(augment_data(data, 10), random=False)

In [5]:
trainer = Trainer(max_epochs=1000)
for model in models:
    epochs = trainer.fit(model, train_dataloader, val_dataloader)

    print(f"{str(model)} done in {epochs} epochs")

Simple perceptron done in 28 epochs
ADALINE done in 28 epochs


In [6]:
for model in models:
    print(f"{str(model)}:")
    for x, _ in data:
        print(f"    {x} -> {model(x)}")

Simple perceptron:
    [[1. 1.]] -> [[1]]
    [[ 1. -1.]] -> [[-1]]
    [[-1. -1.]] -> [[-1]]
    [[-1.  1.]] -> [[-1]]
ADALINE:
    [[1. 1.]] -> [[1]]
    [[ 1. -1.]] -> [[-1]]
    [[-1. -1.]] -> [[-1]]
    [[-1.  1.]] -> [[-1]]
